# Associated Sports Events Column
This Jupyter Notebook will be used to parse the webpages and add data to the column in the master dataset which mentions related sports event based on date

In [8]:
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import csv
import warnings
warnings.filterwarnings('ignore')

Here we define a few functions, parse_link parses the webpages for the sports events tables, and saves those tables in the raw_files folder. The parse_dates function parses the string dates and converts them into date time objects showing the start and end datetime of a particular sports event. Finally, the save_formatted_sports_events_table brings both of these functions together and saves the formatted version.

In [9]:
def parse_link(url, year):
    # make a request to the webpage
    response = requests.get(url)

    # parse the HTML content using Beautiful Soup
    soup = BeautifulSoup(response.content, 'html.parser')

    # find the table containing the events
    table = soup.find('table', {'class': 'list'})

    # find all the rows in the table
    rows = table.find_all('tr')

    # create a CSV file to write the scraped data
    with open(f'raw_files/events_{year}.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # write the table headers to the CSV file
        headers = [th.text.strip() for th in rows[0].find_all('th')]
        writer.writerow(headers)

        # write the table rows to the CSV file, filtering out rows containing "postponed" or "canceled"
        for row in rows[1:]:
            data = [td.text.strip() for td in row.find_all('td')]
            data = [x.replace('posponed', "postponed") for x in data]
            data = [x.replace('postoned', "postponed") for x in data]

            if 'postponed from' in data[0].lower():
                writer.writerow(data)
            elif 'canceled' in data[0].lower() or 'postponed' in data[0].lower():
                continue
            else:
                writer.writerow(data)


def parse_dates(date_str, year):
    date_str = date_str.split(' (')[0]
    date_str = date_str.replace('July', 'Jul')
    date_str = date_str.replace('June', 'Jun')
    date_str = date_str.replace('Sept', 'Sep')
    date_str = date_str.replace('–', '-')
    date_str = date_str.replace('8 Mar', 'Mar 8')
    date_str = date_str.replace('1 Mar', 'Mar 1')

    # split date range into start and end dates
    date_range = date_str.split('-')

    date_range = [x.strip() for x in date_range]

    if len(date_range) == 1:
        # if there is only one date, it is both the start and end date
        date_start = datetime.strptime(date_range[0] + f' {year}', '%b %d %Y')
        date_end = date_start
    else:
        # if second value can be converted to int, this is an individual month
        if len(date_range[-1]) <= 2:
            date_start = datetime.strptime(date_range[0] + f' {year}', '%b %d %Y')
            date_end = datetime(int(year), date_start.month, int(date_range[-1]))
        else:
            # otherwise, parse both start and end dates
            date_start = datetime.strptime(date_range[0] + f' {year}', '%b %d %Y')
            date_end = datetime.strptime(date_range[1] + f' {year}', '%b %d %Y')

    return date_start, date_end

def save_formatted_sports_events_table(url, year):
    parse_link(url, year)
    save_filename = f"raw_files/events_{year}.csv"
    df = pd.read_csv(save_filename)
    df['date_start'], df['date_end'] = zip(*df['Date(s)'].apply(parse_dates, year=year))
    df.to_csv(save_filename, index=False)

Now that we have the functions, we iterate over the years 2020, 2021, and 2022 and save the tables. We then merge these tables into one df, and save that csv.

In [10]:
years = [2020, 2021, 2022]
dfs = list()

for year in years:
    url = f'https://www.topendsports.com/events/calendar-{year}.htm'
    save_formatted_sports_events_table(url, year)

for year in years:
    df = pd.read_csv(f"raw_files/events_{year}.csv")
    dfs.append(df)

total = pd.concat(dfs)
total.drop(columns='Date(s)', inplace=True)
total.to_csv('raw_files/sports_events.csv', index=False)

We now have a fully merged csv with everything we need. We need to convert this into a df which has dates ranging from Jan 1st 2020 to Dec 31st 2022, which we will further merge with our master dataset.

In [11]:
df = pd.read_csv("raw_files/sports_events.csv")
# convert start and end times to datetime
df['date_start'] = pd.to_datetime(df['date_start'])
df['date_end'] = pd.to_datetime(df['date_end'])

# create a range of dates from Jan 1, 2020 to Dec 31, 2022
dates = pd.date_range(start='2020-01-01', end='2022-12-31')

# create an empty dataframe to store the results
result = pd.DataFrame(columns=['Date', 'Event'])

# iterate over each date and find the events that occurred on that date
for date in dates:
    events = df.loc[(df['date_start'] <= date) & (df['date_end'] >= date), 'Event'].tolist()
    event_str = ', '.join(events)
    result = result.append({'Date': date, 'Event': event_str}, ignore_index=True)

# print the resulting dataframe
result.to_csv("sports_events_final.csv", index=False)

We got our df to merge. We finally perform the merging.

In [12]:
df1 = pd.read_csv('../Master Datasets/Master_Dataset_Raw.csv')
df2 = pd.read_csv("sports_events_final.csv")

og_dates = df1['Account Created Date']

# Convert "Account Created Date" column in df1 to datetime
df1['Account Created Date'] = pd.to_datetime(df1['Account Created Date'])

# Convert "Date" column in df2 to datetime
df2['Date'] = pd.to_datetime(df2['Date'])

# merge the two dataframes based on the dates
merged_df = pd.merge(df1, df2[['Date', 'Event']], how='left', left_on=df1['Account Created Date'].dt.date, right_on=df2['Date'].dt.date)

# drop the duplicate date column
merged_df = merged_df.drop('key_0', axis=1)

# rename the 'Event' column to 'Events'
merged_df = merged_df.rename(columns={'Event': 'Events'})

# set the 'Events' column to NaN where there is no matching date
merged_df.loc[merged_df['Events'].isna(), 'Events'] = ''

# drop the duplicate date column
merged_df = merged_df.drop('Date', axis=1)

merged_df = merged_df.rename(columns={'Events': 'Sports Events'})

# display the resulting dataframe
merged_df

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Date (No Timestamp),Interest,Sports Events
0,120776,STY1659352850,310,USR1609427903,male,45,Interview with the Nobel laureate Sir Richard ...,My interview with the Nobel laureate Sir Richa...,https://image.pixstory.com/Pixstory-image-1659...,2020-12-31 20:48:00,2020-12-31,science and technology,
1,127677,STY1660623694,310,USR1609427903,male,45,Participated in the historical Har Ghar Tirang...,'Har Ghar Tiranga’ is a campaign under the aeg...,https://image.pixstory.com/Pixstory-image-1660...,2020-12-31 20:48:00,2020-12-31,nationalism,
2,123275,STY1659941140,8601,USR1630424770,others,13,8. Contemporary Afghanistan : Major Ethnic gro...,The Challenges to Nation-Building\nin Afghanis...,https://image.pixstory.com/Pixstory-image-1630...,2021-08-31 21:16:00,2021-08-31,"contemporary afghanistan, books","Paralympic Games, US Open"
3,121532,STY1659512311,8601,USR1630424770,others,13,6. Contemporary Afghanistan : Gender roles wit...,The Challenges to Nation-Building\nin Afghanis...,https://image.pixstory.com/Pixstory-image-1630...,2021-08-31 21:16:00,2021-08-31,"contemporary afghanistan, books","Paralympic Games, US Open"
4,121141,STY1659422889,8601,USR1630424770,others,13,5. Contemporary Afghanistan : The 'Afghan Pride',The Challenges to Nation-Building\nin Afghanis...,https://image.pixstory.com/Pixstory-image-1630...,2021-08-31 21:16:00,2021-08-31,"contemporary afghanistan, books","Paralympic Games, US Open"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,126771,STY1660496705,14847,USR1641042672,NaN,17,Shared from Adya U Routray,Friendshipday'22,https://image.pixstory.com/Pixstory-image-1660...,2022-01-01 18:41:00,2022-01-01,"friendshipday22, friendshipday'22",
94996,126765,STY1660496512,14846,USR1641042530,NaN,23,On-screen friendships,Indian show Panchayat.,https://image.pixstory.com/Pixstory-image-1660...,2022-01-01 18:38:00,2022-01-01,friendshipday22,
94997,125347,STY1660325731,14837,USR1641025987,NaN,20,Panchayat one of my fav show,Loved the friendship on the show,https://image.pixstory.com/Pixstory-image-1660...,2022-01-01 14:03:00,2022-01-01,friendshipday22,
94998,125429,STY1660360677,14835,USR1641023505,NaN,47,On-screen Bffs,Glad to read this.,https://image.pixstory.com/Pixstory-image-1660...,2022-01-01 13:21:00,2022-01-01,friendshipday,


In [13]:
merged_df.to_csv('../Master Datasets/Master_Dataset_Raw_SportsEvents.csv', index=False)